# Cluster Analysis of Conformere Ensembles

CREST has generated a large number of conformers for a rylene dye.
The dye has 6 substituents of importance; 4 -X-Ph groups (X={O, S, Se}) and 2 butyl groups for solubility.
While the orientation of the -X-Ph groups plays an important role for the spectroscopic properties, the butyl groups are merely for solubility and do not influence spectroscopical properties too much.
We are only interested in different conformers with respect to the -X-Ph groups, but the butyl groups create large numbers of redundant conformers.
Clustering algorithms and PCA is used to clean up the large number of conformers and form groups of similar conformers with respect to the -X-Ph groups.

![Dye Structure](data/structure.png)


In [1]:
-- Enables useful Haskell language extensions.
{-# LANGUAGE TypeApplications #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE RecordWildCards #-}
{-# LANGUAGE FlexibleContexts #-}

-- Import Haskell libraries. 
-- ConClusion provides parsers and toolings to work with 
-- conformere trajectories and statistical tools.
import RIO
import qualified RIO.Text as Text
import qualified RIO.Seq as Seq
import Data.Attoparsec.Text hiding (D)
import ConClusion.Chemistry.Topology
import ConClusion.Numeric.Statistics
import qualified Data.Massiv.Array as Massiv
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Backend.Cairo
import qualified Data.IntSet as IntSet

Line 2: Unused LANGUAGE pragma
Found:
{-# LANGUAGE TypeApplications #-}
Why not:
Line 3: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:
Line 4: Unused LANGUAGE pragma
Found:
{-# LANGUAGE RecordWildCards #-}
Why not:

## Trajectory Parsing and Processing
The trajectory is parsed into a sequence of cartesian molecular structures.
From the geometries, a set of features can be calculated, that is suitable to discriminate the conformers of interest.
In this case we focus on 8 dihedral angles (2 per -X-Ph group), that describe the rotation around the C-X bonds.

In [2]:
-- Defines the 8 dihedral angle features, that we are interested in.
dihedFeatures = 
  [ -- Group 1
    Dihedral (D 16 15 0 34),
    Dihedral (D 15 0 35 43),
    -- Group 2
    Dihedral (D 9 8 7 95),
    Dihedral (D 8 7 95 104),
    -- Group 3
    Dihedral (D 32 31 6 84),
    Dihedral (D 31 6 84 85),
    -- Group 4
    Dihedral (D 27 28 5 73),
    Dihedral (D 18 5 73 82)
  ]

-- Read and parse the trajectory
trj <- readFileUtf8 "data/crest_conformers.xyz" 
  >>= return . parseOnly trajectory >>= \t -> case t of
    Left err -> throwString $ "Could not parse trajectory. Error message: " <> err
    Right res -> return res

-- Calculate the feature matrix from the trajectory, using the defined features.
featureMat <- Massiv.compute @Massiv.U <$> getFeatures dihedFeatures trj

## Statistical Preprocessing
The feature matrix should not be directly processed as is.
Potentially completely different metrics have been combined (with different numerical ranges and units), and we can assume that the features are not completely independent from each other and couple to some extent (as the rotation of the -X-Ph groups sometimes hinder each other).
Therefore three important preprocessing steps need to happen:
  1. The feature matrix must be brought to mean deviation form
  2. The mean deviation must be normalised to bring all components into the range of [-1 .. 1]
  3. A Principal Component Analysis of the feature matrix reduces the dimensionalty from 16 (dihedrals actually need to features to form a valid metric) and removes couplings

In [3]:
-- The `pca` function takes care of all three steps above and returns a comprehensive record type.
-- We keep just 2 dimensions
PCA{..} <- pca 2 featureMat

-- Print information about the PCA.
runSimpleApp . logInfo $
    "PCA RESULTS:\n\
    \  mean squared error: " <> display mse <> "\n\
    \  captured behaviour: " <> display remaining <> "\n\
    \  eigenvalues       : " <> (displayShow . Massiv.toList $ allEigenValues)


PCA RESULTS:
  mean squared error: 4.90148545185776
  captured behaviour: 24.999566449797378
  eigenvalues       : [0.8695180202436779,0.7652486655491315,0.7173978415236926,0.6669662931179737,0.578260723434105,0.5016918915774834,0.39819260913832183,0.39629338481830295,0.3645107503806045,0.26073994389025495,0.2438114410409753,0.22228773338625513,0.2094834071159551,0.16589707997392086,9.767265527007267e-2,8.120770522559635e-2]

## Clustering
ConClusion provides 2 robust clustering algorithms, which do not depend on randomness or guesses (such as k-means): DBScan and Hierarchical Clustering.
Both have a small set of parametres, that can be tuned, and both work on a distance matrix, that somehow describes the distances between (PCA-)features.
Euclidean distances are the most common choice, but we could also use a general L_P norm or advanced distance metrics such as Mahalanobis distances.
We will use DBScan and Euclidean distances here, for its speed, robustness and the fact, that we can easily exclude too small clusters.

In [5]:
-- Use DBScan to find clusters in the PCA-features.
clusters <- dbscan
  euclidean -- Euclidean distances
  5         -- minimal size of a cluster
  0.1       -- search radius in the given distance metric
  y         -- PCA feature matrix

-- Plot the clustering results. Needs tuples of coordinates,
-- therefore we look up the clustered groups in the feature matrix.
pcaFeaturePerConf = Massiv.compute @Massiv.B . Massiv.innerSlices $ y
clusterGroups <- forM clusters $ \gr -> do
  cg <- traverse (pcaFeaturePerConf Massiv.!?) (IntSet.toAscList gr)
  return . fmap (\c -> let cc = Massiv.compute @Massiv.B c in (cc Massiv.! 0, cc Massiv.! 1)) $ cg

toFile def "Clusters.png" $ do
   layout_title .= "DBScan Clusters"
   
   forM_ clusterGroups $ \cg -> plot (points "clusters" cg)

![Cluster analysis"](Clusters.png)